In [95]:
from collections import OrderedDict
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
sns.set()

In [96]:
def getContinent(target,target_num):
    # Targets Continent
    # 2 - 95 NA   
    #100 - 165 SA  
    # 200 - 360 Europe 
    #165 -173 Asia -1 
    # 404 - 625 Africa 
    # 630 - 698 MEast
    # 630 - 910 Asia 
    # ME
    continent=""
    if 2<=target_num<=95:
        continent="North America"
    elif 100<=target_num<=165:
        continent="South America"
    elif 200<=target_num<=390 and target_num!=365:
        continent="Europe"
    elif 404<=target_num<=625:
        continent="Africa"
    elif 630<=target_num<=698:
        continent="Middle East"
    elif 900<=target_num<=940:
        continent="Australia & Oceania"
    elif 699<=target_num<=910 or 165<=target_num<=173 or target_num==365 or target_num==1:
        continent="Asia"
    
    return continent
 
def init_target():
    target=OrderedDict()
    for continent in continents:
        target[continent]=[0,0]
    target['No Support']=0
    return target
    

In [139]:
file="pipeFormat.csv"
continents=["Europe","Asia","Middle East","Africa","North America","South America","Australia & Oceania"]
for nameTarget in continents:
    f=open(file,"r")
    headers=f.readline()
    headers=headers.split("|")
    data=OrderedDict()
    nagid={}
    targets=init_target()
    start,end=1980,2011
    act,defrac,none=0,0,0
    temp={}
    for l in f:
        row=l.split("|")
        year=int(row[3])
        target=row[8]
        support=row[38]

        nagname=row[11]
        
        try:
            target_num=int(row[headers.index('tarnum_gw')])
        except:
            target_num=-1
        try:
            support_num=int(row[headers.index('supnum_gw')])
        except:
            support_num=-1
        active=int(row[headers.index('statesup')])
        defracto=int(row[headers.index('defacto')])

        supportContinent=getContinent(support,support_num)
        key=supportContinent
        if supportContinent not in temp:
            temp[supportContinent]=set()
        temp[supportContinent].add(support+'-'+str(support_num))
        if getContinent(target,target_num)=="Australia & Oceania":
            print(year, target,target_num, support,nagname)
        if nameTarget==getContinent(target,target_num):
            if start<=year<end:
                if key!='':
                    if key not in targets:
                        print("error",key,targets.keys())
                    if active==1:
                        targets[key][0]+=1
                    if defracto==1:
                        targets[key][1]+=1
                if target_num!=-1:
                    if support_num==-1 or support_num==0:
                        none+=1
                   
                   
                        
    targets['No Support']=none                 
    print(nameTarget,targets,none)
    tempfname='pickles/{}-{}-{}.pickle'.format(start,end,nameTarget.replace(' ',''))
    #pickle.dump(targets,open(tempfname,"wb"))

1988 Papua New Guinea 910 Singapore BRA
1988 Papua New Guinea 910 Solomon Islands BRA
1989 Papua New Guinea 910 Solomon Islands BRA
1989 Papua New Guinea 910 Singapore BRA
1990 Papua New Guinea 910 Singapore BRA
1990 Papua New Guinea 910 Solomon Islands BRA
1991 Papua New Guinea 910 Singapore BRA
1991 Papua New Guinea 910 Solomon Islands BRA
1992 Papua New Guinea 910 Solomon Islands BRA
1992 Papua New Guinea 910 Singapore BRA
1993 Papua New Guinea 910 Singapore BRA
1993 Papua New Guinea 910 Solomon Islands BRA
1994 Papua New Guinea 910 Solomon Islands BRA
1994 Papua New Guinea 910 Singapore BRA
1995 Papua New Guinea 910 Singapore BRA
1995 Papua New Guinea 910 Solomon Islands BRA
1996 Papua New Guinea 910 Singapore BRA
1996 Papua New Guinea 910 Solomon Islands BRA
1997 Papua New Guinea 910 Solomon Islands BRA
1997 Papua New Guinea 910 Singapore BRA
Europe OrderedDict([('Europe', [36, 128]), ('Asia', [43, 29]), ('Middle East', [12, 9]), ('Africa', [25, 0]), ('North America', [2, 30]), ('

In [140]:
for nameTarget in ["Europe","Asia","Middle East","Africa","North America","South America","Australia & Oceania"]:
    a,b=1945,1980
    c,d=1980,2011
    prefn='pickles/{}-{}-{}.pickle'.format(a,b,nameTarget.replace(' ',''))
    postfn='pickles/{}-{}-{}.pickle'.format(c,d,nameTarget.replace(' ',''))
    pre=pickle.load(open(prefn,"rb"))
    post=pickle.load(open(postfn,"rb"))
    preNoSupport=pre['No Support']
    postNoSupport=post['No Support']
    del pre['No Support']
    del post['No Support']
    
    plt.close()
    plt.figure(figsize = (8,4), dpi = 200)
    gs = gridspec.GridSpec(1,4)
    maxRange=max([pre[i][0] for i in pre],[pre[i][1] for i in pre],[post[i][0] for i in post],[post[i][1] for i in post])
    maxRange=max(maxRange)+50
    
    plt.subplot(gs[0,0:2])
    print(pre)
    
    plt.barh(-1,preNoSupport,color='k',height=0.3)
    plt.barh([i for i in range(len(pre))],[pre[i][0] for i in pre],color='b',height=0.3)
    plt.barh([i-0.3 for i in range(len(pre))],[pre[i][1] for i in pre],color='g',height=0.3)
    yticks=[-1]+[i for i in range(len(pre))]
    yticksLabels=['No Support']+[i for i in pre]
    plt.yticks(yticks,yticksLabels)
    plt.xlabel("Number of Observations")
   
    plt.title("Pre-1980")
    plt.xlim(-1,maxRange)
    
    ax=plt.subplot(gs[0,2:4])
    print(post)
    plt.barh(-1,postNoSupport,color='k',height=0.3)
    plt.barh([i for i in range(len(post))],[post[i][0] for i in post],color='b',height=0.3,label="Active Support")
    plt.barh([i-0.3 for i in range(len(pre))],[post[i][1] for i in post],color='g',height=0.3,label="Defacto Support")
    plt.legend(loc='upper right')
    ax.yaxis.set_major_formatter(plt.NullFormatter())
    plt.title("Post-1980")
    plt.xlim(-1,maxRange)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.80)
    plt.suptitle("Support Distribution of NAGs targeting {}".format(nameTarget))
    plt.xlabel("Number of Observations")
    plt.savefig("images/{}.png".format(nameTarget.replace(' ','')),bbox_inches='tight')
    

OrderedDict([('Europe', [13, 26]), ('Asia', [139, 7]), ('Middle East', [12, 19]), ('Africa', [89, 0]), ('North America', [34, 11]), ('South America', [0, 0]), ('Australia & Oceania', [0, 0])])
OrderedDict([('Europe', [36, 128]), ('Asia', [43, 29]), ('Middle East', [12, 9]), ('Africa', [25, 0]), ('North America', [2, 30]), ('South America', [0, 11]), ('Australia & Oceania', [0, 0])])
OrderedDict([('Europe', [3, 105]), ('Asia', [271, 228]), ('Middle East', [10, 0]), ('Africa', [0, 8]), ('North America', [13, 9]), ('South America', [0, 0]), ('Australia & Oceania', [2, 5])])
OrderedDict([('Europe', [43, 162]), ('Asia', [506, 966]), ('Middle East', [87, 115]), ('Africa', [4, 25]), ('North America', [43, 105]), ('South America', [0, 0]), ('Australia & Oceania', [0, 75])])
OrderedDict([('Europe', [26, 109]), ('Asia', [77, 0]), ('Middle East', [227, 353]), ('Africa', [43, 113]), ('North America', [34, 35]), ('South America', [0, 0]), ('Australia & Oceania', [0, 0])])
OrderedDict([('Europe', [7

In [56]:
temp

{'': {'--1', '-0', '.--1', '0--1', '0-0', 'Malaysia--1'},
 'Africa': {'-530',
  'Algeria-615',
  'Angola-500',
  'Angola-540',
  'Benin-434',
  'Botswana-571',
  'Burkina Faso-439',
  'Burundi-516',
  'Cameroon-471',
  'Central African Republic-482',
  'Chad-483',
  'Congo, Democratic Republic of (Zaire)-490',
  'Congo-484',
  'Democratic Republic of Congo (Zaire)-490',
  'Democratic Republic of Congo-490',
  'Djibouti-522',
  'Djibouti-552',
  'Eritrea-531',
  'Ethiopia-530',
  'Gambia-420',
  'Ghana-452',
  'Guinea-438',
  'Guinea-Bissau-404',
  'Ivory Coast-437',
  'Kenya-501',
  'Liberia-450',
  'Libya-620',
  'Libya-625',
  'Mali-432',
  'Mauritania-435',
  'Morocco-600',
  'Mozambique-541',
  'Namibia-565',
  'Niger-436',
  'Nigeria-475',
  'Rwanda-517',
  'Senegal-433',
  'Sierra Leone-451',
  'Somali-520',
  'Somalia-520',
  'South Africa-560',
  'Sudan-625',
  'Tanzania-510',
  'Tunisia-616',
  'Uganda-500',
  'Zaire-490',
  'Zambia-551',
  'Zimbabwe-552'},
 'Asia': {'Afghanis

In [8]:
headers

['triadid',
 'UCDPdyadid',
 'conflictid',
 'year',
 'yr_active',
 'foundyr',
 'startdate1',
 'startdate2',
 'target',
 'tarnum_cow',
 'tarnum_gw',
 'nag_name',
 'nagcode_1',
 'nagcode_2',
 'incomp',
 'terr',
 'nagid_1',
 'nagid_2',
 'nagid_3',
 'nagid_4',
 'nagid_5',
 'nageth',
 'nagrel',
 'nagleft',
 'nagright',
 'nagdem',
 'nagauth',
 'nagdict',
 'nagmil',
 'nagtheo',
 'nagobj_1',
 'nagobj_2',
 'nagobj_3',
 'nagobj_4',
 'nagobj_5',
 'nagobj_6',
 'polpardummy',
 'partyname',
 'supporter',
 'supnum_gw',
 'supnum_COW',
 'statesup',
 's_safemem',
 's_precision_sm',
 's_safelead',
 's_precision_sl',
 's_headquar',
 's_precision_head',
 's_traincamp',
 's_precision_tc',
 's_training',
 's_precision_train',
 's_weaponlog',
 's_precision_wl',
 's_finaid',
 's_precision_finaid',
 's_transport',
 's_precision_tp',
 's_troop',
 's_precision_tr',
 's_other',
 's_precision_oth',
 'defacto',
 'ds_safemem',
 'ds_precision_sm',
 'ds_safelead',
 'ds_precision_sl',
 'ds_headquar',
 'ds_precision_head'

In [124]:
max(5,10,40)

40